In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from functools import partial

import torch.nn.functional as F
from torch import nn
import torch
import emmental
from emmental import Meta
from emmental.learner import EmmentalLearner
from emmental.model import EmmentalModel
from emmental.scorer import Scorer
from emmental.task import EmmentalTask
from modules.bert_module import BertModule
from parse_COPA import get_COPA_dataloaders
from task_config import SuperGLUE_LABEL_MAPPING, SuperGLUE_TASK_METRIC_MAPPING
from sklearn.metrics import f1_score

In [3]:
logger = logging.getLogger(__name__)

# Initalize Emmental

In [4]:
emmental.init(
    "logs",
    config={
        "meta_config": {"seed": 111},
        "model_config": {"device": 1, "dataparallel": False},
        "learner_config": {
            "n_epochs": 10,
            "valid_split": "val",
            "optimizer_config": {"optimizer": "adam", "lr": 1e-5},
            "lr_scheduler_config": {
#                 "warmup_percentage": 0.1,
                "lr_scheduler": None, #"linear",
                "min_lr": 1e-7,
            },
        },
        "logging_config": {
            "counter_unit": "batch",
            "evaluation_freq": 100,
            "checkpointing": None,
        },
    },
)

[2019-05-28 14:38:28,481][INFO] emmental.meta:95 - Setting logging directory to: logs/2019_05_28/14_38_28
[2019-05-28 14:38:28,499][INFO] emmental.meta:56 - Loading Emmental default config from /dfs/scratch1/senwu/mmtl/emmental/src/emmental/emmental-default-config.yaml.
[2019-05-28 14:38:28,500][INFO] emmental.meta:143 - Updating Emmental config from user provided config.


In [5]:
Meta.config

{'meta_config': {'seed': 111, 'verbose': True, 'log_path': None},
 'model_config': {'model_path': None, 'device': 1, 'dataparallel': False},
 'learner_config': {'fp16': False,
  'n_epochs': 10,
  'train_split': 'train',
  'valid_split': 'val',
  'test_split': 'test',
  'ignore_index': 0,
  'optimizer_config': {'optimizer': 'adam',
   'lr': 1e-05,
   'l2': 0.0,
   'grad_clip': 1.0,
   'sgd_config': {'momentum': 0.9},
   'adam_config': {'betas': (0.9, 0.999)}},
  'lr_scheduler_config': {'lr_scheduler': None,
   'warmup_steps': None,
   'warmup_unit': 'batch',
   'warmup_percentage': None,
   'min_lr': 1e-07,
   'linear_config': {'min_lr': 0.0},
   'exponential_config': {'gamma': 0.9},
   'plateau_config': {'factor': 0.5, 'patience': 10, 'threshold': 0.0001}},
  'task_scheduler': 'round_robin',
  'global_evaluation_metric_dict': None},
 'logging_config': {'counter_unit': 'batch',
  'evaluation_freq': 100,
  'writer_config': {'writer': 'tensorboard', 'verbose': True},
  'checkpointing': No

In [6]:
TASK_NAME = "COPA"
DATA_DIR = "data"
BERT_MODEL_NAME = "bert-large-cased"
BATCH_SIZE = 4

BERT_OUTPUT_DIM = 768 if "base" in BERT_MODEL_NAME else 1024

# Extract train/dev dataset from file

In [7]:
dataloaders = get_COPA_dataloaders(
    data_dir=DATA_DIR,
    task_name=TASK_NAME,
    splits=["train", "val", "test"],
    max_sequence_length=256,
    max_data_samples=None,
    tokenizer_name=BERT_MODEL_NAME,
    batch_size=BATCH_SIZE,
)

[2019-05-28 14:38:28,634][INFO] tokenizer:8 - Loading Tokenizer bert-large-cased
[2019-05-28 14:38:28,903][INFO] pytorch_pretrained_bert.tokenization:190 - loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased-vocab.txt from cache at /lfs/local/0/senwu/.pytorch_pretrained_bert/cee054f6aafe5e2cf816d2228704e326446785f940f5451a5b26033516a4ac3d.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


data/COPA/train.jsonl
{'premise': 'My body cast a shadow over the grass.', 'choice1': 'The sun was rising.', 'choice2': 'The grass was cut.', 'question': 'cause', 'label': 0, 'idx': 0}


[2019-05-28 14:38:29,274][INFO] parse_COPA:123 - Loaded train for COPA.
[2019-05-28 14:38:29,359][INFO] parse_COPA:123 - Loaded val for COPA.


max len 34
data/COPA/val.jsonl
{'premise': 'The man turned on the faucet.', 'choice1': 'The toilet filled with water.', 'choice2': 'Water flowed from the spout.', 'question': 'effect', 'label': 1, 'idx': 0}
max len 34
data/COPA/test.jsonl
{'premise': 'The item was packaged in bubble wrap.', 'choice1': 'It was fragile.', 'choice2': 'It was small.', 'question': 'cause', 'idx': 0}


[2019-05-28 14:38:29,755][INFO] parse_COPA:123 - Loaded test for COPA.


max len 36


# Build Emmental task

In [8]:
def ce_loss(task_name, immediate_ouput_dict, Y, active):
    module_name = f"{task_name}_pred_head"
    return F.cross_entropy(
        immediate_ouput_dict[module_name][0][active], (Y.view(-1) - 1)[active]
    )

In [9]:
def output(task_name, immediate_ouput_dict):
    module_name = f"{task_name}_pred_head"
    return F.softmax(immediate_ouput_dict[module_name][0], dim=1)

In [11]:
class LinearModule(nn.Module):
    def __init__(self, n_features, n_class):
        super().__init__()

        self.linear = nn.Linear(n_features, n_class)

    def forward(self, input):
        return self.linear.forward(input[-1][:,0,:])

In [12]:
class ChoiceModule(nn.Module):
    def __init__(self, n_choices=2):
        super().__init__()

        self.n_choices = n_choices

    def forward(self, immediate_ouput_dict):
        logits = []
        for i in range(self.n_choices):
            logits.append(immediate_ouput_dict[f"choice{str(i)}rep"][0])

        logits = torch.cat(logits, dim=1)

        return logits

In [14]:
emmental_task = EmmentalTask(
    name=TASK_NAME,
    module_pool=nn.ModuleDict(
        {
            "bert_module": BertModule(BERT_MODEL_NAME),
            "linear_module": LinearModule(BERT_OUTPUT_DIM, 1),
            f"{TASK_NAME}_pred_head": ChoiceModule(2),
        }
    ),
    task_flow=[
        {
            "name": "choice0",
            "module": "bert_module",
            "inputs": [("_input_", "token1_ids")],
        },
        {
            "name": "choice1",
            "module": "bert_module",
            "inputs": [("_input_", "token2_ids")],
        },
        {
            "name": "choice0rep",
            "module": "linear_module",
            "inputs": [("choice0", 0)],
        },
        {
            "name": "choice1rep",
            "module": "linear_module",
            "inputs": [("choice1", 0)],
        },        
        {
            "name": f"{TASK_NAME}_pred_head",
            "module": f"{TASK_NAME}_pred_head",
            "inputs": [],
        },
    ],
    loss_func=partial(ce_loss, TASK_NAME),
    output_func=partial(output, TASK_NAME),
    scorer=Scorer(
        metrics=SuperGLUE_TASK_METRIC_MAPPING[TASK_NAME]
    ),
)

[2019-05-28 14:38:30,272][INFO] pytorch_pretrained_bert.modeling:580 - loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-cased.tar.gz from cache at ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233
[2019-05-28 14:38:30,275][INFO] pytorch_pretrained_bert.modeling:588 - extracting archive file ./cache/7fb0534b83c42daee7d3ddb0ebaa81387925b71665d6ea195c5447f1077454cd.eea60d9ebb03c75bb36302aa9d241d3b7a04bba39c360cf035e8bf8140816233 to temp dir /tmp/tmpqmfpvqd2
[2019-05-28 14:38:47,809][INFO] pytorch_pretrained_bert.modeling:598 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "max_position_embeddings": 512,
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pooler_fc_size": 768,
  "pooler_num_a

In [16]:
mtl_model = EmmentalModel(name="SuperGLUE_single_task", tasks=[emmental_task])

[2019-05-28 14:39:19,578][INFO] emmental.model:58 - Moving model to GPU (cuda:1).
[2019-05-28 14:39:25,832][INFO] emmental.model:44 - Created emmental model SuperGLUE_single_task that contains task {'COPA'}.
[2019-05-28 14:39:25,835][INFO] emmental.model:58 - Moving model to GPU (cuda:1).


In [17]:
emmental_learner = EmmentalLearner()

In [18]:
emmental_learner.learn(mtl_model, dataloaders.values())

[2019-05-28 14:39:25,928][INFO] emmental.logging.logging_manager:33 - Evaluating every 100 batch.
[2019-05-28 14:39:25,929][INFO] emmental.logging.logging_manager:51 - No checkpointing.
[2019-05-28 14:39:25,980][INFO] root:123 - Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
[2019-05-28 14:39:26,030][INFO] root:123 - Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
[2019-05-28 14:39:26,173][INFO] emmental.learner:299 - Start learning...


In [19]:
mtl_model.score(dataloaders["val"])

{'COPA/SuperGLUE/val/accuracy': 0.63}

In [20]:
mtl_model.score(dataloaders["train"])

{'COPA/SuperGLUE/train/accuracy': 1.0}